In [4]:
import os
import itertools
import csv

In [5]:
# Crear directorio SiData si no existe
directorio = ".in"
os.makedirs(directorio, exist_ok=True)

# Definiendo parámetros de entrada

In [6]:
# Parámetros para iterar
elemento = 'Au'  # Puedes cambiar este elemento manualmente
ecuts = range(10, 101,5)  # Energía de corte de 10 a 100 Ry de 10 en 10
kpuntos = [(x, x, x) for x in range(1, 12)]  # k points de (1,1,1) a (11,11,11) de 2 en 2
pseudopotenciales = ['Au.pbe-n-kjpaw_psl.1.0.0', 'Au.pbe-n-rrkjus_psl.1.0.0', 'Au.pbesol-n-kjpaw_psl.1.0.0', 'Au.pbesol-n-rrkjus_psl.1.0.0']     

In [7]:
plantilla = """
&control
  calculation='scf',
  prefix='{elemento}',
  tstress = .true.,
  tprnfor = .true.,
  outdir='./out_{elemento}',
  pseudo_dir='/home/lorenzo/pseudos/Au',
/
&system
  ibrav=2,
  celldm(1)= 7.71,
  nat=1, 
  ntyp=1,
  ecutwfc={ecut},
  occupations='smearing', 
  smearing='gaussian',  
  degauss=0.01,    
/
&electrons
  diagonalization='david',
  conv_thr=1.0d-8,
/
ATOMIC_SPECIES
  {elemento}  196.966  {pseudopotencial}.UPF
ATOMIC_POSITIONS {{angstrom}}
  {elemento} 0.00 0.00 0.00 
K_POINTS {{automatic}}
  {kx} {ky} {kz} 0 0 0
"""

In [8]:
# Nombre del archivo CSV actualizado para generarlo en el mismo bucle
nombre_csv = os.path.join("AuData.csv")

In [9]:
# Crear y llenar el archivo CSV con la información de los archivos generados
with open(nombre_csv, mode='w', newline='') as csv_file:
    fieldnames = ['Archivo', 'Elemento', 'Ecut', 'KPoints', 'Pseudopotencial']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    # Generar archivos .in y registrarlos en el CSV
    for ecut, (kx, ky, kz), pseudopotencial in itertools.product(ecuts, kpuntos, pseudopotenciales):
        nombre_archivo = f"{elemento}_ecut{ecut}_k{kx}{ky}{kz}_{pseudopotencial}.in"
        path_completo = os.path.join(directorio, nombre_archivo)
        with open(path_completo, 'w') as archivo:
            contenido = plantilla.format(
                elemento=elemento,
                ecut=ecut,
                kx=kx, ky=ky, kz=kz,
                pseudopotencial=pseudopotencial
            )
            archivo.write(contenido)
        
        # Añadir información al CSV
        writer.writerow({
            'Archivo': nombre_archivo,
            'Elemento': elemento,
            'Ecut': ecut,
            'KPoints': f"{kx} {ky} {kz}",
            'Pseudopotencial': pseudopotencial
        })